In [2]:
import pandas as pd
import glob
import os

In [17]:
path = ('/Users/paolo/Documents/TC/Paolo/data/Indices/')
all_files = glob.glob(os.path.join(path, "*.csv"))

df = pd.read_csv(all_files[0])
for f in all_files[1:]:
    aux_df = pd.read_csv(f)
    df = pd.concat((df,aux_df.iloc[:,1:]), axis=1)
df

,Unnamed: 0,NAOP_S,BLOCK_S,NAOM_S,RIDGE_S,nino3,nino4,nino34,nino12,indocW,...,BLOCK,NAOM,RIDGE,EAS,Arctic,weu,NAO_classic,WeMO,PNA,NPD
0,1980-01-01,-0.182057,-1.804146,0.690041,1.425914,23.954093,28.309878,26.591961,24.046144,28.036842,...,-1.091872,0.916174,0.858326,-3.510473,39.113934,0.364284,5.982506,NaN,6.380449,0.070947
1,1980-01-02,-0.333924,-0.904377,0.195294,1.178619,23.982840,28.305696,26.569124,24.003577,28.076732,...,-0.628396,0.457480,0.697619,-4.316831,31.318435,0.364769,16.567497,NaN,4.327160,-0.372845
2,1980-01-03,-0.382689,0.013145,-0.074517,0.502674,24.053181,28.349518,26.608834,23.893645,28.089537,...,-0.114731,0.089294,0.449448,-4.096059,9.162693,0.365231,28.132500,NaN,0.487942,-1.042550
3,1980-01-04,-0.072198,-0.015091,0.035592,0.045248,24.127478,28.406776,26.680904,23.756035,28.055619,...,-0.132405,0.184506,0.257954,-4.563278,8.043160,0.365614,36.470001,NaN,-2.873871,1.342918
4,1980-01-05,-0.261882,0.014397,0.114259,0.117818,24.184204,28.460211,26.768365,23.657757,27.981568,...,0.191257,0.242473,0.002844,-6.212076,25.560463,0.365948,21.022499,NaN,-4.641118,5.615116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15945,2023-08-28,-0.560067,0.487704,-0.277526,0.405942,26.790646,29.710604,28.244773,24.355470,28.360921,...,0.075940,-0.405470,0.356717,1.888322,13.145353,0.317727,20.385002,-13.519997,1.416902,4.614209
15946,2023-08-29,-0.534366,0.219864,-0.234510,0.637364,26.755699,29.741776,28.258911,24.340400,28.343777,...,-0.055382,-0.319646,0.424297,1.590642,20.408620,0.317635,10.308746,-16.712502,2.652396,3.251957
15947,2023-08-30,-0.241968,0.276627,-0.285007,0.311772,26.657818,29.795078,28.304199,24.270261,28.330869,...,0.005388,-0.279854,0.204001,1.038108,9.354852,0.317551,4.352501,-8.687500,1.843450,0.805065
15948,2023-08-31,-0.029180,0.304703,-0.163869,-0.114591,26.499222,29.792601,28.270638,24.195576,28.241875,...,0.044327,-0.168327,0.019747,-0.953342,7.679418,0.317450,5.427505,-5.736252,-1.793373,-0.245677


In [21]:
# WeMO index removed since it has some Nan and time renamed
df = df.dropna(axis=1).rename(columns={'Unnamed: 0':'time'})
df

,time,NAOP_S,BLOCK_S,NAOM_S,RIDGE_S,nino3,nino4,nino34,nino12,indocW,...,NAOP,BLOCK,NAOM,RIDGE,EAS,Arctic,weu,NAO_classic,PNA,NPD
0,1980-01-01,-0.182057,-1.804146,0.690041,1.425914,23.954093,28.309878,26.591961,24.046144,28.036842,...,-0.698796,-1.091872,0.916174,0.858326,-3.510473,39.113934,0.364284,5.982506,6.380449,0.070947
1,1980-01-02,-0.333924,-0.904377,0.195294,1.178619,23.982840,28.305696,26.569124,24.003577,28.076732,...,-0.541367,-0.628396,0.457480,0.697619,-4.316831,31.318435,0.364769,16.567497,4.327160,-0.372845
2,1980-01-03,-0.382689,0.013145,-0.074517,0.502674,24.053181,28.349518,26.608834,23.893645,28.089537,...,-0.415616,-0.114731,0.089294,0.449448,-4.096059,9.162693,0.365231,28.132500,0.487942,-1.042550
3,1980-01-04,-0.072198,-0.015091,0.035592,0.045248,24.127478,28.406776,26.680904,23.756035,28.055619,...,-0.293791,-0.132405,0.184506,0.257954,-4.563278,8.043160,0.365614,36.470001,-2.873871,1.342918
4,1980-01-05,-0.261882,0.014397,0.114259,0.117818,24.184204,28.460211,26.768365,23.657757,27.981568,...,-0.359942,0.191257,0.242473,0.002844,-6.212076,25.560463,0.365948,21.022499,-4.641118,5.615116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15945,2023-08-28,-0.560067,0.487704,-0.277526,0.405942,26.790646,29.710604,28.244773,24.355470,28.360921,...,-0.071530,0.075940,-0.405470,0.356717,1.888322,13.145353,0.317727,20.385002,1.416902,4.614209
15946,2023-08-29,-0.534366,0.219864,-0.234510,0.637364,26.755699,29.741776,28.258911,24.340400,28.343777,...,-0.100789,-0.055382,-0.319646,0.424297,1.590642,20.408620,0.317635,10.308746,2.652396,3.251957
15947,2023-08-30,-0.241968,0.276627,-0.285007,0.311772,26.657818,29.795078,28.304199,24.270261,28.330869,...,0.030442,0.005388,-0.279854,0.204001,1.038108,9.354852,0.317551,4.352501,1.843450,0.805065
15948,2023-08-31,-0.029180,0.304703,-0.163869,-0.114591,26.499222,29.792601,28.270638,24.195576,28.241875,...,0.084769,0.044327,-0.168327,0.019747,-0.953342,7.679418,0.317450,5.427505,-1.793373,-0.245677


In [22]:
# Save as unique file
df.to_csv('/Users/paolo/Documents/TC/Paolo/data/Indices/full_indices.csv',index=False)

In [24]:
pd.read_csv('/Users/paolo/Documents/TC/Paolo/data/Indices/full_indices.csv')

,time,NAOP_S,BLOCK_S,NAOM_S,RIDGE_S,nino3,nino4,nino34,nino12,indocW,...,NAOP,BLOCK,NAOM,RIDGE,EAS,Arctic.1,weu.2,NAO_classic,PNA,NPD
0,1980-01-01,-0.182057,-1.804146,0.690041,1.425914,23.954093,28.309878,26.591961,24.046144,28.036842,...,-0.698796,-1.091872,0.916174,0.858326,-3.510473,39.113934,0.364284,5.982506,6.380449,0.070947
1,1980-01-02,-0.333924,-0.904377,0.195294,1.178619,23.982840,28.305696,26.569124,24.003577,28.076732,...,-0.541367,-0.628396,0.457480,0.697619,-4.316831,31.318435,0.364769,16.567497,4.327160,-0.372845
2,1980-01-03,-0.382689,0.013145,-0.074517,0.502674,24.053181,28.349518,26.608834,23.893645,28.089537,...,-0.415616,-0.114731,0.089294,0.449448,-4.096059,9.162693,0.365231,28.132500,0.487942,-1.042550
3,1980-01-04,-0.072198,-0.015091,0.035592,0.045248,24.127478,28.406776,26.680904,23.756035,28.055619,...,-0.293791,-0.132405,0.184506,0.257954,-4.563278,8.043160,0.365614,36.470001,-2.873871,1.342918
4,1980-01-05,-0.261882,0.014397,0.114259,0.117818,24.184204,28.460211,26.768365,23.657757,27.981568,...,-0.359942,0.191257,0.242473,0.002844,-6.212076,25.560463,0.365948,21.022499,-4.641118,5.615116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15945,2023-08-28,-0.560067,0.487704,-0.277526,0.405942,26.790646,29.710604,28.244773,24.355470,28.360921,...,-0.071530,0.075940,-0.405470,0.356717,1.888322,13.145353,0.317727,20.385002,1.416902,4.614209
15946,2023-08-29,-0.534366,0.219864,-0.234510,0.637364,26.755699,29.741776,28.258911,24.340400,28.343777,...,-0.100789,-0.055382,-0.319646,0.424297,1.590642,20.408620,0.317635,10.308746,2.652396,3.251957
15947,2023-08-30,-0.241968,0.276627,-0.285007,0.311772,26.657818,29.795078,28.304199,24.270261,28.330869,...,0.030442,0.005388,-0.279854,0.204001,1.038108,9.354852,0.317551,4.352501,1.843450,0.805065
15948,2023-08-31,-0.029180,0.304703,-0.163869,-0.114591,26.499222,29.792601,28.270638,24.195576,28.241875,...,0.084769,0.044327,-0.168327,0.019747,-0.953342,7.679418,0.317450,5.427505,-1.793373,-0.245677


### Add as additional columns of Sindian data

In [3]:
train = pd.read_csv('/Users/paolo/Documents/TC/Paolo/data/mutiple_regions/train_dailymeans_Natlantic.csv')
train

,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,sst,tcwv,tclw,tciw,shear
0,0,40.0,-90.0,1980-01-01,-0.000007,44.713528,10.698517,3.193596,-6.865570,-5.106041,0.00000,7.027856,0.035780,0.005125,7.708423
1,1,40.0,-87.5,1980-01-01,0.000007,36.624430,9.591637,1.696613,-3.868561,-4.458462,0.00000,6.300080,0.044869,0.010488,7.917031
2,2,40.0,-85.0,1980-01-01,0.000007,37.621235,10.684670,0.584248,-1.069183,-2.137562,0.00000,6.600942,0.046887,0.008923,10.156769
3,3,40.0,-82.5,1980-01-01,0.000003,19.087340,12.444183,0.113667,1.313889,-0.631775,0.00000,6.244626,0.028166,0.009826,12.483078
4,4,40.0,-80.0,1980-01-01,-0.000001,12.242437,14.768517,0.392281,2.876091,0.841381,0.00000,5.439828,0.004323,0.008743,14.519511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268766,4268766,10.0,-30.0,2010-12-31,0.000003,5.591190,23.921143,-10.037945,-7.237564,0.686100,300.55978,27.328184,0.026579,0.000000,34.871250
4268767,4268767,10.0,-27.5,2010-12-31,0.000004,10.311977,25.225800,-10.199718,-10.804726,1.613747,300.39606,29.313910,0.013027,0.000002,37.539124
4268768,4268768,10.0,-25.0,2010-12-31,0.000005,46.144000,25.110214,-10.788048,-14.048752,1.782543,299.82794,31.990301,0.013662,0.000063,39.234104
4268769,4268769,10.0,-22.5,2010-12-31,-0.000003,61.234660,22.956375,-10.748108,-15.813065,1.396748,300.43832,33.541737,0.010098,0.000061,37.844020


In [33]:
train_withIndex = train.merge(df, on='time', how='left')
train_withIndex

,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,...,NAOP,BLOCK,NAOM,RIDGE,EAS,Arctic,weu,NAO_classic,PNA,NPD
0,0,0.0,20.0,1980-01-01,0.000007,80.761185,1.909660,-3.323872,1.687164,-1.823624,...,-0.698796,-1.091872,0.916174,0.858326,-3.510473,39.113934,0.364284,5.982506,6.380449,0.070947
1,1,0.0,22.5,1980-01-01,0.000004,80.703650,1.165733,-2.844494,1.060593,-1.991425,...,-0.698796,-1.091872,0.916174,0.858326,-3.510473,39.113934,0.364284,5.982506,6.380449,0.070947
2,2,0.0,25.0,1980-01-01,0.000007,78.231514,-1.311676,-2.125244,3.280617,-1.931789,...,-0.698796,-1.091872,0.916174,0.858326,-3.510473,39.113934,0.364284,5.982506,6.380449,0.070947
3,3,0.0,27.5,1980-01-01,0.000010,79.631010,-3.777573,-1.122395,5.743889,-1.243538,...,-0.698796,-1.091872,0.916174,0.858326,-3.510473,39.113934,0.364284,5.982506,6.380449,0.070947
4,4,0.0,30.0,1980-01-01,0.000010,71.573875,-5.734505,-1.362953,6.514030,-0.954163,...,-0.698796,-1.091872,0.916174,0.858326,-3.510473,39.113934,0.364284,5.982506,6.380449,0.070947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4268766,4268766,-30.0,80.0,2010-12-31,0.000015,26.797535,25.075424,-3.653679,-1.221291,1.515594,...,-2.074062,-0.001751,1.279814,1.038660,-7.434265,69.713316,0.350066,-7.794998,-2.316406,-2.969247
4268767,4268767,-30.0,82.5,2010-12-31,-0.000006,27.766910,24.175919,-2.866638,-6.724304,0.861771,...,-2.074062,-0.001751,1.279814,1.038660,-7.434265,69.713316,0.350066,-7.794998,-2.316406,-2.969247
4268768,4268768,-30.0,85.0,2010-12-31,0.000010,29.111805,24.655510,-2.809170,-10.138817,0.051220,...,-2.074062,-0.001751,1.279814,1.038660,-7.434265,69.713316,0.350066,-7.794998,-2.316406,-2.969247
4268769,4268769,-30.0,87.5,2010-12-31,0.000006,27.833050,25.088104,-2.730087,-11.036507,0.666927,...,-2.074062,-0.001751,1.279814,1.038660,-7.434265,69.713316,0.350066,-7.794998,-2.316406,-2.969247


In [36]:
train_withIndex.to_csv('/Users/paolo/Documents/TC/Paolo/data/multiple_regions_withIndices/train_dailymeans_Sindian.csv',index=False)


In [37]:
val = pd.read_csv('/Users/paolo/Documents/TC/Paolo/data/mutiple_regions/val_dailymeans_Sindian.csv')
val

,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,sst,tcwv,tclw,tciw,shear
0,0,0.0,20.0,2011-01-01,0.000003,73.016390,-5.760780,-4.216808,6.860649,-4.352928,0.00000,45.908466,0.051140,0.010802,11.319371
1,1,0.0,22.5,2011-01-01,0.000003,74.569660,-4.942451,-3.857407,6.459419,-3.991157,0.00000,45.768530,0.054900,0.014612,10.506754
2,2,0.0,25.0,2011-01-01,0.000004,80.080090,-3.848740,-3.175144,6.303680,-3.446140,0.00000,47.055485,0.085401,0.041099,9.773062
3,3,0.0,27.5,2011-01-01,0.000012,83.676704,0.330811,-2.526569,7.235268,-2.307594,0.00000,42.947980,0.091923,0.081013,9.961468
4,4,0.0,30.0,2011-01-01,0.000011,76.225440,3.678749,-1.027561,7.020271,-0.077572,0.00000,32.404747,0.098393,0.042447,8.516380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688397,688397,-30.0,80.0,2015-12-31,0.000014,70.662056,23.560066,1.655861,9.690376,3.621418,296.77530,33.177513,0.011262,0.000468,22.729418
688398,688398,-30.0,82.5,2015-12-31,-0.000006,51.213654,22.381706,0.321705,9.860390,-0.099480,296.44290,30.789354,0.029158,0.000318,24.204187
688399,688399,-30.0,85.0,2015-12-31,0.000009,46.678970,22.464828,0.851299,7.661758,-0.725330,295.73486,30.004541,0.039861,0.000132,23.183786
688400,688400,-30.0,87.5,2015-12-31,0.000002,59.362090,22.364807,0.543045,5.595253,-1.542034,295.24792,31.301584,0.131010,0.001035,22.959316


In [38]:
val_withIndex = val.merge(df, on='time', how='left')
val_withIndex

,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,...,NAOP,BLOCK,NAOM,RIDGE,EAS,Arctic,weu,NAO_classic,PNA,NPD
0,0,0.0,20.0,2011-01-01,0.000003,73.016390,-5.760780,-4.216808,6.860649,-4.352928,...,-2.015218,-0.341078,1.251136,1.288358,-6.327494,70.721592,0.350303,-13.502502,-3.838239,-1.283170
1,1,0.0,22.5,2011-01-01,0.000003,74.569660,-4.942451,-3.857407,6.459419,-3.991157,...,-2.015218,-0.341078,1.251136,1.288358,-6.327494,70.721592,0.350303,-13.502502,-3.838239,-1.283170
2,2,0.0,25.0,2011-01-01,0.000004,80.080090,-3.848740,-3.175144,6.303680,-3.446140,...,-2.015218,-0.341078,1.251136,1.288358,-6.327494,70.721592,0.350303,-13.502502,-3.838239,-1.283170
3,3,0.0,27.5,2011-01-01,0.000012,83.676704,0.330811,-2.526569,7.235268,-2.307594,...,-2.015218,-0.341078,1.251136,1.288358,-6.327494,70.721592,0.350303,-13.502502,-3.838239,-1.283170
4,4,0.0,30.0,2011-01-01,0.000011,76.225440,3.678749,-1.027561,7.020271,-0.077572,...,-2.015218,-0.341078,1.251136,1.288358,-6.327494,70.721592,0.350303,-13.502502,-3.838239,-1.283170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688397,688397,-30.0,80.0,2015-12-31,0.000014,70.662056,23.560066,1.655861,9.690376,3.621418,...,1.858815,1.378519,-0.949125,-2.265438,-4.520714,45.358388,0.326075,63.114998,3.556290,0.967407
688398,688398,-30.0,82.5,2015-12-31,-0.000006,51.213654,22.381706,0.321705,9.860390,-0.099480,...,1.858815,1.378519,-0.949125,-2.265438,-4.520714,45.358388,0.326075,63.114998,3.556290,0.967407
688399,688399,-30.0,85.0,2015-12-31,0.000009,46.678970,22.464828,0.851299,7.661758,-0.725330,...,1.858815,1.378519,-0.949125,-2.265438,-4.520714,45.358388,0.326075,63.114998,3.556290,0.967407
688400,688400,-30.0,87.5,2015-12-31,0.000002,59.362090,22.364807,0.543045,5.595253,-1.542034,...,1.858815,1.378519,-0.949125,-2.265438,-4.520714,45.358388,0.326075,63.114998,3.556290,0.967407


In [39]:
val_withIndex.to_csv('/Users/paolo/Documents/TC/Paolo/data/multiple_regions_withIndices/val_dailymeans_Sindian.csv',index=False)


In [40]:
test = pd.read_csv('/Users/paolo/Documents/TC/Paolo/data/mutiple_regions/test_dailymeans_Sindian.csv')
test

,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,sst,tcwv,tclw,tciw,shear
0,0,0.0,20.0,2016-04-01,0.000004,81.055560,-8.377182,5.273445,2.527817,2.160847,0.00000,53.959347,0.066320,0.052278,13.655559
1,1,0.0,22.5,2016-04-01,-0.000002,76.221700,-6.143539,4.284462,4.278809,2.030232,0.00000,53.494244,0.086551,0.236994,10.667676
2,2,0.0,25.0,2016-04-01,0.000018,75.465416,-1.724731,3.315552,3.811935,1.344738,0.00000,53.025135,0.314800,0.518474,5.611731
3,3,0.0,27.5,2016-04-01,-0.000009,72.850200,1.577347,1.549385,4.243774,2.951813,0.00000,48.393536,0.186379,0.423175,1.292264
4,4,0.0,30.0,2016-04-01,-0.000009,82.229460,1.983276,1.535709,5.736717,0.056183,0.00000,41.883312,0.094952,0.209450,5.698139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929677,929677,-30.0,80.0,2022-12-31,0.000008,14.955483,13.163338,-4.289364,4.153252,-2.868565,295.33643,24.162498,0.033585,0.001839,18.812302
929678,929678,-30.0,82.5,2022-12-31,0.000023,10.061256,13.957420,-4.323608,7.508942,-0.484577,295.08510,23.452278,0.044212,0.001561,19.952251
929679,929679,-30.0,85.0,2022-12-31,0.000009,8.901871,15.958916,-3.855770,11.119324,2.067028,294.75940,19.158775,0.035525,0.000474,21.784533
929680,929680,-30.0,87.5,2022-12-31,-0.000002,8.630913,19.073456,-3.618012,14.113441,2.629562,294.41754,17.160767,0.043179,0.001137,25.431913


In [41]:
test_withIndex = test.merge(df, on='time', how='left')
test_withIndex

,Unnamed: 0,latitude,longitude,time,vo,r,u_200,u_850,v_200,v_850,...,NAOP,BLOCK,NAOM,RIDGE,EAS,Arctic,weu,NAO_classic,PNA,NPD
0,0,0.0,20.0,2016-04-01,0.000004,81.055560,-8.377182,5.273445,2.527817,2.160847,...,1.297699,0.131767,-0.552550,-0.975293,-2.094453,-13.830361,0.359531,36.680000,-0.956713,9.263693
1,1,0.0,22.5,2016-04-01,-0.000002,76.221700,-6.143539,4.284462,4.278809,2.030232,...,1.297699,0.131767,-0.552550,-0.975293,-2.094453,-13.830361,0.359531,36.680000,-0.956713,9.263693
2,2,0.0,25.0,2016-04-01,0.000018,75.465416,-1.724731,3.315552,3.811935,1.344738,...,1.297699,0.131767,-0.552550,-0.975293,-2.094453,-13.830361,0.359531,36.680000,-0.956713,9.263693
3,3,0.0,27.5,2016-04-01,-0.000009,72.850200,1.577347,1.549385,4.243774,2.951813,...,1.297699,0.131767,-0.552550,-0.975293,-2.094453,-13.830361,0.359531,36.680000,-0.956713,9.263693
4,4,0.0,30.0,2016-04-01,-0.000009,82.229460,1.983276,1.535709,5.736717,0.056183,...,1.297699,0.131767,-0.552550,-0.975293,-2.094453,-13.830361,0.359531,36.680000,-0.956713,9.263693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929677,929677,-30.0,80.0,2022-12-31,0.000008,14.955483,13.163338,-4.289364,4.153252,-2.868565,...,1.527989,-0.652236,0.102924,-1.111479,-3.912794,3.183185,0.319780,30.979996,3.079765,-4.957011
929678,929678,-30.0,82.5,2022-12-31,0.000023,10.061256,13.957420,-4.323608,7.508942,-0.484577,...,1.527989,-0.652236,0.102924,-1.111479,-3.912794,3.183185,0.319780,30.979996,3.079765,-4.957011
929679,929679,-30.0,85.0,2022-12-31,0.000009,8.901871,15.958916,-3.855770,11.119324,2.067028,...,1.527989,-0.652236,0.102924,-1.111479,-3.912794,3.183185,0.319780,30.979996,3.079765,-4.957011
929680,929680,-30.0,87.5,2022-12-31,-0.000002,8.630913,19.073456,-3.618012,14.113441,2.629562,...,1.527989,-0.652236,0.102924,-1.111479,-3.912794,3.183185,0.319780,30.979996,3.079765,-4.957011


In [42]:
test_withIndex.to_csv('/Users/paolo/Documents/TC/Paolo/data/multiple_regions_withIndices/test_dailymeans_Sindian.csv',index=False)


In [43]:
df.columns

Index(['time', 'NAOP_S', 'BLOCK_S', 'NAOM_S', 'RIDGE_S', 'nino3', 'nino4',
       'nino34', 'nino12', 'indocW', 'indocE', 'AtlMDR', 'CarMDR', 'GulfMDR',
       'AtlSub', 'glob', '30_40N', 'weu', 'EOF1', 'EOF2', 'EOF3', 'Arctic',
       'eq', '60N', '60S', 'weu', 'eeu', 'cus', 'NAO', 'EAT', 'ABLOCK', 'NAOP',
       'BLOCK', 'NAOM', 'RIDGE', 'EAS', 'Arctic', 'weu', 'NAO_classic', 'PNA',
       'NPD'],
      dtype='object')